In [ ]:
from transformers import BartTokenizer, BartForQuestionAnswering
import torch

tokenizer = BartTokenizer.from_pretrained('a-ware/bart-squadv2')
model = BartForQuestionAnswering.from_pretrained('a-ware/bart-squadv2')

question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
encoding = tokenizer(question, text, return_tensors='pt')
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

start_scores, end_scores = model(input_ids, attention_mask=attention_mask, output_attentions=False)[:2]

all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])
answer = tokenizer.convert_tokens_to_ids(answer.split())
answer = tokenizer.decode(answer)
answer
#answer => 'a nice puppet'

In [3]:
answer

'<s>Who was Jim Henson?</s></s>Jim Henson was a nice'

In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
# see ``examples/summarization/bart/evaluate_cnn.py`` for a longer example
model = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa')
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')

# inputs = tokenizer.batch_encode_plus([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')
# # Generate Summary
# summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=5, early_stopping=True)
# print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

In [ ]:
ARTICLE_TO_SUMMARIZE = "Computer vision is an interdisciplinary field that deals with how computers can be made to gain high-level understanding from digital images or videos. From the perspective of engineering, it seeks to automate tasks that the human visual system can do. \"Computer vision is concerned with the automatic extraction, analysis and understanding of useful information from a single image or a sequence of images. It involves the development of a theoretical and algorithmic basis to achieve automatic visual understanding.\" As a scientific discipline, computer vision is concerned with the theory behind artificial systems that extract information from images. The image data can take many forms, such as video sequences, views from multiple cameras, or multi-dimensional data from a medical scanner. As a technological discipline, computer vision seeks to apply its theories and models for the construction of computer vision systems."

query = f'question: What is computer vision? context: <P> {ARTICLE_TO_SUMMARIZE}'

inputs = tokenizer([query], max_length=1024, return_tensors='pt')

# Generate Summary
ids = model.generate(inputs['input_ids'], num_beams=8, min_length=20, max_length=256,
                                           do_sample=False,
                                           early_stopping=True,
                                           temperature=1.0,
                                           top_k=None,
                                           top_p=None,
                                           eos_token_id=tokenizer.eos_token_id,
                                           no_repeat_ngram_size=3,
                                           num_return_sequences=1)
answer=tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0  ]
print(answer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.

KeyboardInterrupt



In [5]:
xq = model.encode([query]).tolist()
xc = index.query(xq, top_k=1, include_mtadata=True)
xc

AttributeError: 'BartForConditionalGeneration' object has no attribute 'encode'

In [34]:
ARTICLE_TO_SUMMARIZE = "Computer vision is an interdisciplinary field that deals with how computers can be made to gain high-level understanding from digital images or videos. From the perspective of engineering, it seeks to automate tasks that the human visual system can do. \"Computer vision is concerned with the automatic extraction, analysis and understanding of useful information from a single image or a sequence of images. It involves the development of a theoretical and algorithmic basis to achieve automatic visual understanding.\" As a scientific discipline, computer vision is concerned with the theory behind artificial systems that extract information from images. The image data can take many forms, such as video sequences, views from multiple cameras, or multi-dimensional data from a medical scanner. As a technological discipline, computer vision seeks to apply its theories and models for the construction of computer vision systems."

query = f'question: Who is Boris Johnson? context: <P> {ARTICLE_TO_SUMMARIZE}'

inputs = tokenizer([query], max_length=1024, return_tensors='pt')

# Generate Summary
ids = model.generate(inputs['input_ids'], num_beams=2, min_length=0, max_length=256,
                                           do_sample=False,
                                           early_stopping=True,
                                           temperature=1.0,
                                           top_p=0.95, top_k=50,
                                           eos_token_id=tokenizer.eos_token_id,
                                           no_repeat_ngram_size=3,
                                           num_return_sequences=1, repetition_penalty=2.0)
answer=tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0  ]
print(answer)

computer vision researcher


In [22]:
from scipy import spatial  # for calculating vector similarities for search
def similarity(query_embedding: list,
                   knowledge_embedding: list
    ) -> float:
        """Calculates the cosine similarity score between the query and knowledge embedding vectors."""

        return 1- spatial.distance.cosine(query_embedding, knowledge_embedding)
from sentence_transformers import SentenceTransformer
similarity_model = SentenceTransformer('bert-base-nli-mean-tokens')
question_encoding = similarity_model.encode('What is computer vision?')
paragraph_encoding = similarity_model.encode(context)
similarity(question_encoding, paragraph_encoding)

0.6217034459114075

In [14]:
from  transformers  import  AutoTokenizer, AutoModelWithLMHead, pipeline

model_name = "MaRiOrOsSi/t5-base-finetuned-question-answering"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name)

C:\Users\point\anaconda3\lib\site-packages\transformers\models\auto\modeling_auto.py:1352: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


C:\Users\point\anaconda3\lib\site-packages\transformers\generation\utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


The answer to life, the universe, and everything in it's entirety.


In [20]:
question = "What are tensors?"
context = "Computer vision is an interdisciplinary field that deals with how computers can be made to gain high-level understanding from digital images or videos. From the perspective of engineering, it seeks to automate tasks that the human visual system can do. \"Computer vision is concerned with the automatic extraction, analysis and understanding of useful information from a single image or a sequence of images. It involves the development of a theoretical and algorithmic basis to achieve automatic visual understanding.\" As a scientific discipline, computer vision is concerned with the theory behind artificial systems that extract information from images. The image data can take many forms, such as video sequences, views from multiple cameras, or multi-dimensional data from a medical scanner. As a technological discipline, computer vision seeks to apply its theories and models for the construction of computer vision systems."
input = f"question: {question} context: {context}"
encoded_input = tokenizer([input],
                             return_tensors='pt',
                             max_length=512,
                             truncation=True)
output = model.generate(input_ids = encoded_input.input_ids,
                            attention_mask = encoded_input.attention_mask)
output = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)

C:\Users\point\anaconda3\lib\site-packages\transformers\generation\utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Two dimensional models for the construction of computer vision systems.


In [7]:
from transformers import pipeline

qa_model = pipeline("question-answering")
question = "How can computer vision be applied to engineering?"
context = "Computer vision is an interdisciplinary field that deals with how computers can be made to gain high-level understanding from digital images or videos. From the perspective of engineering, it seeks to automate tasks that the human visual system can do. \"Computer vision is concerned with the automatic extraction, analysis and understanding of useful information from a single image or a sequence of images. It involves the development of a theoretical and algorithmic basis to achieve automatic visual understanding.\" As a scientific discipline, computer vision is concerned with the theory behind artificial systems that extract information from images. The image data can take many forms, such as video sequences, views from multiple cameras, or multi-dimensional data from a medical scanner. As a technological discipline, computer vision seeks to apply its theories and models for the construction of computer vision systems."
qa_model(question = question, context = context)
## {'answer': 'İstanbul', 'end': 39, 'score': 0.953, 'start': 31}

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.



KeyboardInterrupt



In [35]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


SyntaxError: 'return' outside function (4096520225.py, line 19)

In [47]:
from transformers import pipeline

def generate_answer(prompt, context):
    """
    Generate an answer to a question given a context using GPT-2.

    Args:
        prompt (str): The question prompt.
        context (str): The context in which the question is being asked.

    Returns:
        str: The generated answer.
    """
    model_name = "gpt2"
    nlp = pipeline("question-answering", model=model_name)
    qa_input = {
        "question": prompt,
        "context": context
    }
    result = nlp(qa_input)
    return result['answer']
generate_answer('How can computer vision be applied to engineering?', context)

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at gpt2 and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
print(question)

What are tensors?
